Preprocessing - **PCA** (component=4)  
Algorithm - **KNN Classification**  
Feature - **Magnitude** **(251)**

In [5]:
# Real + Imaginary together as one feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:4000]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

trainData - (3200, 15, 251, 2)
testData - (800, 15, 251, 2)


In [6]:
# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  # Now shape is (samples, 251*2) -> (12800, 502)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)  # (samples, 251*2) -> (12800, 502)
    print(f"data_scaled: {data_scaled.shape}")
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

# -------- Train --------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # (6400, 251, 2)
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # (6400, 251, 2)

train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])  # (6400, 251)
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])        # (6400, 251)
train_cirs = np.vstack((train_alice_magnitude, train_eve_magnitude))  # (12800, 251)

# Apply PCA
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp)  # (12800, n_components)

# Labels
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # '0' for Alice
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # '1' for Eve
train_labels = np.hstack((train_alice_labels, train_eve_labels))  # (12800,)

# -------- Test --------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]  # (1600, 251, 2)
test_eve_cirs = testCIR[:, eve_channel, :, :]      # (1600, 251, 2)

test_alice_magnitude = np.abs(test_alice_cirs[..., 0] + 1j * test_alice_cirs[..., 1])  # (1600, 251)
test_eve_magnitude = np.abs(test_eve_cirs[..., 0] + 1j * test_eve_cirs[..., 1])        # (1600, 251)
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))  # (3200, 251)

# Labels
test_alice_label = np.zeros(test_alice_cirs.shape[0])  # '0' for Alice
test_eve_labels = np.ones(test_eve_cirs.shape[0])       # '1' for Eve
test_labels = np.hstack((test_alice_label, test_eve_labels))  # (3200,)

# Reshape and Scale
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)  # (3200, 251*2)
test_cirs_scaled = scaler.transform(reshaped_test_cirs)

# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)  # (3200, n_components)


data_scaled: (6400, 251)


In [7]:

# ----------------- Classification -----------------
# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_cirs_pca, train_labels)

# Predict on test data
predictions = knn.predict(test_cirs_pca)


In [8]:
# ----------------- Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 57.19%
tp: 462
tn: 453
fp: 347
fn: 338
MDR: 0.43375
FAR: 0.4225
AR: 0.571875
